In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import numpy as np
import torch.utils.data as data_utils
import random 
from matplotlib import pyplot as plt
import sys
import utilities as ut
from Network import Net
import Network as nn
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

batch_size = 256
hidden_size = 100
num_classes = 2


/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/franciscoperez/anacon

In [2]:
pd.read_csv('13_01_2023_gaussian2d.csv').sort_values('acc_test', ascending=False).head(5)

,Unnamed: 0,acc_train,acc_test,recall_train,recall_test,f1_train,f1_test,roc_train,roc_test,epsilon,batch_size,hidden_size,aux_loss_activated,EPS1,n,opt
26,26,99.876190,82.247920,0.999557,0.888126,0.999159,0.854043,0.999124,0.892750,0,256,100,True,0.025,50000,1
9,9,99.929070,82.139850,0.999246,0.854389,0.999269,0.837569,0.999373,0.879369,0,256,100,True,0.025,50000,1
20,20,99.907120,82.115570,0.999734,0.863371,0.999403,0.841734,0.999259,0.876956,0,256,100,True,0.025,50000,1
2,2,99.867546,81.307396,0.999558,0.872864,0.999117,0.841909,0.998952,0.873357,0,256,100,True,0.025,50000,1
15,15,99.884705,81.280590,0.999601,0.849131,0.999224,0.830572,0.999119,0.870377,0,256,100,True,0.025,50000,1


In [3]:
pd.read_csv('13_01_2023_baseline.csv').sort_values('acc_test', ascending=False).head(5)

,Unnamed: 0,acc_train,acc_test,recall_train,recall_test,f1_train,f1_test,roc_train,roc_test,epsilon,batch_size,hidden_size,aux_loss_activated,EPS1,n,opt
26,26,99.889800,81.44302,0.999162,0.881992,0.999030,0.846866,0.999148,0.875753,0.2,256,100,False,0.025,50000,2
18,18,99.902610,79.44064,0.999601,0.898131,0.999314,0.843090,0.999293,0.866602,0.2,256,100,False,0.025,50000,2
3,3,99.893870,79.36625,0.999469,0.865123,0.999204,0.827854,0.999059,0.855890,0.2,256,100,False,0.025,50000,2
11,11,99.907110,79.35332,0.999425,0.883526,0.999248,0.836115,0.999115,0.863383,0.2,256,100,False,0.025,50000,2
15,15,99.920395,79.10737,0.999646,0.864612,0.999425,0.826210,0.999324,0.856055,0.2,256,100,False,0.025,50000,2


In [4]:
datContent = [i.strip().split() for i in open("../paper2-pending/subclasses/ident1.dat").readlines()]

data = pd.DataFrame(datContent)
data.columns = ['ID', 'field_a','field_b', 'sub_clase', 'ccx', 'ccy', 'd', 'e', 'f', 'g' ]
data=data[['ID', 'field_a','field_b', 'sub_clase', 'ccx', 'ccy']]
data.shape

datContent = [i.strip().split() for i in open("../paper2-pending/subclasses/ident2.dat").readlines()]

data2 = pd.DataFrame(datContent)
data2.columns = ['ID', 'field_a','field_b', 'sub_clase', 'ccx', 'ccy']
data = data.append(data2)
data.shape

datContent = [i.strip().split() for i in open("../paper2-pending/subclasses/ident3.dat").readlines()]

data3 = pd.DataFrame(datContent)
data3.columns = ['ID', 'field_a','field_b', 'sub_clase', 'ccx', 'ccy', 'd', 'e', 'f', 'g' ,'k']
data3=data3[['ID', 'field_a','field_b', 'sub_clase', 'ccx', 'ccy']]
data = data.append(data3)
data.shape

datContent = [i.strip().split() for i in open("../paper2-pending/subclasses/ident4.dat").readlines()]

data4 = pd.DataFrame(datContent)
data4.columns = ['ID', 'field_a','field_b', 'sub_clase', 'ccx', 'ccy', 'd', 'e', 'f', 'g' ]
data4=data4[['ID', 'field_a','field_b', 'sub_clase', 'ccx', 'ccy']]
data = data.append(data4)
data.shape


data = data[data.sub_clase.isin(['RRab', 'RRd', 'RRc', 'RRe'])]


In [5]:
def filter_by_subclass(data_subclasses, subclass='RRab'): 
    path_test = '/home/franciscoperez/Documents/GitHub/data/BIASEDFATS/Test_rrlyr-1.csv'
    lc_test = pd.read_table(path_test, sep= ',')
    del lc_test['Unnamed: 0']
    columns_to_export = lc_test.columns
    lc_test['ID'] = lc_test['ID'].str.replace('.dat', '') 
    lc_test = lc_test.merge(data_subclasses, left_on='ID', right_on='ID', how = 'left')
    lc_test = lc_test[lc_test.sub_clase==subclass]
    return lc_test[columns_to_export]

RRab_test = filter_by_subclass(data, subclass='RRab')
RRc_test = filter_by_subclass(data, subclass='RRc')
RRd_test = filter_by_subclass(data, subclass='RRd')
RRe_test = filter_by_subclass(data, subclass='RRe')

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [6]:
train_dataset, test_dataset = ut.load_files(dataset=1)
train_dataset.head()

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0


,Amplitude,AndersonDarling,Autocor_length,Beyond1Std,CAR_mean,CAR_sigma,CAR_tau,Con,Eta_e,FluxPercentileRatioMid20,...,Skew,SlottedA_length,SmallKurtosis,Std,StetsonK,StetsonK_AC,StructureFunction_index_21,StructureFunction_index_31,StructureFunction_index_32,label
49643,0.0435,0.131166,3,0.326667,23.686214,0.738367,0.594257,0.003356,77.820389,0.191176,...,0.104165,0.12345,-0.098630,0.022427,0.816054,0.779717,1.644229,2.212875,1.435421,ClassB
407286,0.0205,0.457675,3,0.270000,54.028838,0.549494,0.244528,0.003356,50.714320,0.171429,...,-0.195980,2.96080,0.590754,0.010581,0.781576,0.702489,1.650383,2.080388,1.284226,ClassB
106019,0.0305,0.842306,4,0.258065,18.163004,0.668703,0.655423,0.000000,6.988632,0.120690,...,0.115857,7.18960,0.061644,0.015522,0.792800,0.772128,1.760809,2.357977,1.384309,ClassB
269038,0.0175,0.113156,2,0.350000,35.016108,0.405370,0.435593,0.006711,36098.082750,0.137931,...,-0.096889,4.04488,0.298335,0.009146,0.793050,0.734278,1.879023,2.721291,1.499985,ClassB
333084,0.0150,0.007341,1,0.273973,34.429653,0.733305,0.449642,0.000000,74.510326,0.130435,...,-0.178951,0.11075,0.838739,0.007203,0.770452,0.706294,1.735545,2.338960,1.432837,ClassB


In [7]:
RRe_test[train_dataset.columns].head()

,Amplitude,AndersonDarling,Autocor_length,Beyond1Std,CAR_mean,CAR_sigma,CAR_tau,Con,Eta_e,FluxPercentileRatioMid20,...,Skew,SlottedA_length,SmallKurtosis,Std,StetsonK,StetsonK_AC,StructureFunction_index_21,StructureFunction_index_31,StructureFunction_index_32,label
16,0.16750,0.018003,1,0.303333,26.158132,0.073465,0.725973,0.0,12.579577,0.150000,...,0.341465,1.96797,0.198594,0.078668,0.790933,0.664890,2.078430,3.183539,1.561949,ClassA
27,0.22550,0.011021,1,0.296667,129.958025,0.513429,0.148121,0.0,145609.518248,0.178660,...,0.460641,0.01650,0.551941,0.126372,0.822407,0.766745,2.038494,3.106660,1.544758,ClassA
164,0.14700,0.037791,1,0.351171,40.168090,0.114340,0.468277,0.0,1161.629293,0.154762,...,0.234766,0.04988,-0.267603,0.079077,0.827947,0.863374,1.854792,2.734859,1.555903,ClassA
352,0.22600,0.224821,1,0.313333,37.115196,0.683457,0.522459,0.0,87.191250,0.171662,...,0.334666,1.95611,0.296205,0.115745,0.805037,0.666054,1.565646,1.875291,1.336562,ClassA
365,0.19975,0.084151,1,0.295302,140.819375,0.270730,0.135191,0.0,140.681435,0.148485,...,0.551426,0.20806,1.326243,0.100128,0.797293,0.784578,1.978843,2.951971,1.585350,ClassA


In [8]:
num_classes = 2
learning_rate = 0.005
samples = 3000
epsilon=0
results = []
#for epsilon in [0.1, 0.05, 0.025, 0.15]:
for batch_size in [256]:
    for hidden_size in [100]:
        for aux_loss_activated in [True, False]:
            for EPS1 in [0.025]:
                for n in [50000]:
                    for opt in [1]:
                        for t in range(10):
                            train_dataset, test_dataset = ut.load_files(dataset=1)
                            test_dataseta  = RRab_test[train_dataset.columns]
                            print(test_dataset.shape)
                            test_dataset = pd.concat([test_dataseta[test_dataseta.label=='ClassB'], test_dataseta])
                            print(test_dataset.shape)
                            test_dataset['random'] = np.random.random(test_dataset.shape[0])
                            test_dataset = test_dataset.sort_values('random')
                            del test_dataset['random']
                            input_size = train_dataset.shape[1]-1
                            train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)
                            if n < 50000:
                                train_dataset = ut.down_sampling(train_dataset)
                                train_dataset = train_dataset.sample(n)
                            else: 
                                trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                n2 = n - trainig_dataset_a.shape[0]
                                trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])
                            
                            train_dataset = ut.sort_columns(train_dataset)
                            test_dataset = ut.sort_columns(test_dataset)
                            test_dataset_pred = test_dataset.copy()
                            train_dataset_pred = train_dataset.copy()

                            data_prior = ut.generate_samples_2D(samples, train_dataset, distribution='gaussian')
                            if train_dataset[train_dataset.label=='ClassB'].shape[0] >= samples:
                                samples_prior = samples 
                            else: 
                                samples_prior = train_dataset[train_dataset.label=='ClassB'].shape[0] 

                            data_prior = pd.concat([data_prior, train_dataset[train_dataset.label=='ClassB'].sample(samples_prior)])


                            train_dataset, test_dataset, data_prior = ut.normalize(train_dataset, test_dataset, data_prior)

                            train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2)

                            train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2)
                            _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                            _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                            _, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
                            _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
                            _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

                            net = Net(input_size, hidden_size, hidden_size, num_classes)
                            net.cuda()

                            hist_val, hist_train, _ = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                            EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated, model_number=t, size=n)

                            acc_train, recall_train, f1_train = nn.get_results(net, train_loader, input_size)
                            acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
                            roc_train = nn.get_roc_curve(net, train_loader, input_size)
                            roc_test = nn.get_roc_curve(net, test_loader, input_size)
                            results.append([acc_train, acc_test,recall_train, recall_test, f1_train, f1_test, roc_train, roc_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt])

                            pd.DataFrame(results, columns=['acc_train', 'acc_test','recall_train', 'recall_test','f1_train', 'f1_test', 
                                   'roc_train', 'roc_test', 'epsilon', 'batch_size', 'hidden_size',
                             'aux_loss_activated', 'EPS1', 'n', 'opt']).to_csv('RRab_17_01_2023_subclasses_gaussian2d.csv')

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0
(28625, 61)
(8175, 61)
____get_tensor_function____
shape tensor:  torch.Size([4800, 60])
____get_tensor_function____
shape tensor:  torch.Size([1201, 60])
____get_tensor_function____
shape tensor:  torch.Size([40000, 60])
____get_tensor_function____
shape tensor:  torch.Size([50000, 60])
____get_tensor_function____
shape tensor:  torch.Size([1201, 60])
____get_tensor_function____
shape tensor:  torch.Size([7779, 60])
____get_tensor_function____
shape tensor:  torch.Size([7779, 60])


/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


training: epoch:  1  loss:  55.31181713938713 -- aux loss:  12.8681600689888
training: epoch:  2  loss:  50.68006780743599 -- aux loss:  11.26086476445198
training: epoch:  3  loss:  50.49630033969879 -- aux loss:  15.60795733332634
training: epoch:  4  loss:  50.41345092654228 -- aux loss:  8.50998169183731
training: epoch:  5  loss:  50.353882133960724 -- aux loss:  8.438534557819366
training: epoch:  6  loss:  50.32209497690201 -- aux loss:  7.929449677467346
training: epoch:  7  loss:  50.34791135787964 -- aux loss:  9.707759648561478
training: epoch:  8  loss:  50.22933980822563 -- aux loss:  7.367851048707962
training: epoch:  9  loss:  50.23355835676193 -- aux loss:  6.8035189509391785
training: epoch:  10  loss:  50.234985530376434 -- aux loss:  6.865957826375961
the_last_loss:  0.0
running_loss_val:  0.34291544556617737
validating: epoch:  10  loss:  0.34291544556617737
The current loss: 0.34291544556617737
the_last_loss: 0.0
trigger times: 1
sum mask2 - L1:  tensor(4799, devi

training: epoch:  67  loss:  49.51362207531929 -- aux loss:  6.060617208480835
training: epoch:  68  loss:  49.51307433843613 -- aux loss:  6.1396417915821075
training: epoch:  69  loss:  49.512084275484085 -- aux loss:  6.146182596683502
training: epoch:  70  loss:  49.49605077505112 -- aux loss:  6.058685153722763
the_last_loss:  0.33070108294487
running_loss_val:  0.3154873251914978
validating: epoch:  70  loss:  0.3154873251914978
The current loss: 0.3154873251914978
the_last_loss: 0.33070108294487
trigger times: 0
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 - L2:  tensor(7553, device='cuda:0')
sum mask2 - L3:  tensor(151, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1201, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2447, device='cuda:0')
sum mask1 - L3 (aux):  tensor(49, device='cuda:0')
training: epoch:  71  loss:  49.53608164191246 -- aux loss:  6.312272429466248
training: epoch:  72  loss:  49.503575295209885 -- aux loss:  6.034740567207336
training: epoch:  7

training: epoch:  131  loss:  49.414283722639084 -- aux loss:  5.975222080945969
training: epoch:  132  loss:  49.3851113319397 -- aux loss:  6.025577425956726
training: epoch:  133  loss:  49.365787357091904 -- aux loss:  5.984868407249451
training: epoch:  134  loss:  49.37168130278587 -- aux loss:  5.999543726444244
training: epoch:  135  loss:  49.44443389773369 -- aux loss:  5.999437481164932
training: epoch:  136  loss:  49.407061725854874 -- aux loss:  6.0286173820495605
training: epoch:  137  loss:  49.37589627504349 -- aux loss:  6.011619567871094
training: epoch:  138  loss:  49.41946169734001 -- aux loss:  5.997465401887894
training: epoch:  139  loss:  49.4143206179142 -- aux loss:  6.03023424744606
training: epoch:  140  loss:  49.3808736205101 -- aux loss:  6.201944410800934
the_last_loss:  0.31326156854629517
running_loss_val:  0.3146775960922241
validating: epoch:  140  loss:  0.3146775960922241
The current loss: 0.3146775960922241
the_last_loss: 0.31326156854629517
tri

...

training: epoch:  1334  loss:  49.29160684347153 -- aux loss:  5.951970964670181
training: epoch:  1335  loss:  49.28890374302864 -- aux loss:  5.951970964670181
training: epoch:  1336  loss:  49.291494369506836 -- aux loss:  5.951970964670181
training: epoch:  1337  loss:  49.28921088576317 -- aux loss:  5.951970964670181
training: epoch:  1338  loss:  49.30595490336418 -- aux loss:  5.951970964670181
training: epoch:  1339  loss:  49.292356848716736 -- aux loss:  5.951970964670181
training: epoch:  1340  loss:  49.294190406799316 -- aux loss:  5.951970964670181
the_last_loss:  0.31891128420829773
running_loss_val:  0.31326156854629517
validating: epoch:  1340  loss:  0.31326156854629517
The current loss: 0.31326156854629517
the_last_loss: 0.31891128420829773
trigger times: 0
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 - L2:  tensor(7553, device='cuda:0')
sum mask2 - L3:  tensor(151, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1201, device='cuda:0')
sum mask1 - L2 (aux

training: epoch:  1397  loss:  49.28774678707123 -- aux loss:  5.951970964670181
training: epoch:  1398  loss:  49.30132406949997 -- aux loss:  5.951971650123596
training: epoch:  1399  loss:  49.28921639919281 -- aux loss:  5.9573363065719604
training: epoch:  1400  loss:  49.29957517981529 -- aux loss:  5.959830701351166
the_last_loss:  0.31326156854629517
running_loss_val:  0.31326156854629517
validating: epoch:  1400  loss:  0.31326156854629517
The current loss: 0.31326156854629517
the_last_loss: 0.31326156854629517
trigger times: 1
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 - L2:  tensor(7553, device='cuda:0')
sum mask2 - L3:  tensor(151, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1201, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2447, device='cuda:0')
sum mask1 - L3 (aux):  tensor(49, device='cuda:0')
training: epoch:  1401  loss:  49.31060978770256 -- aux loss:  5.953454554080963
training: epoch:  1402  loss:  49.31864219903946 -- aux loss:  5.95197099447250

training: epoch:  1460  loss:  49.292927980422974 -- aux loss:  5.951998084783554
the_last_loss:  0.31326156854629517
running_loss_val:  0.31326156854629517
validating: epoch:  1460  loss:  0.31326156854629517
The current loss: 0.31326156854629517
the_last_loss: 0.31326156854629517
trigger times: 7
sum mask2 - L1:  tensor(4799, device='cuda:0')
sum mask2 - L2:  tensor(7553, device='cuda:0')
sum mask2 - L3:  tensor(151, device='cuda:0')
sum mask1 - L1 (aux):  tensor(1201, device='cuda:0')
sum mask1 - L2 (aux):  tensor(2447, device='cuda:0')
sum mask1 - L3 (aux):  tensor(49, device='cuda:0')
training: epoch:  1461  loss:  49.33111613988876 -- aux loss:  5.952334016561508
training: epoch:  1462  loss:  49.300769209861755 -- aux loss:  5.9519723653793335
training: epoch:  1463  loss:  49.29300621151924 -- aux loss:  5.955202519893646
training: epoch:  1464  loss:  49.29470458626747 -- aux loss:  5.953115910291672
training: epoch:  1465  loss:  49.29141807556152 -- aux loss:  5.951979041099

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:943: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning)


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
results = []

num_classes = 2
learning_rate = 0.005
samples = 3000
epsilon=0
#for epsilon in [0.1, 0.05, 0.025, 0.15]:
for batch_size in [256]:
    for hidden_size in [100]:
        for aux_loss_activated in [True, False]:
            for EPS1 in [0.025]:
                for n in [50000]:
                    for opt in [1]:
                        for t in range(10):
                            train_dataset, test_dataset = ut.load_files(dataset=1)
                            test_dataseta  = RRc_test[train_dataset.columns]
                            test_dataset = pd.concat([test_dataseta[test_dataseta.label=='ClassB'], test_dataseta])
                            test_dataset['random'] = np.random.random(test_dataset.shape[0])
                            test_dataset = test_dataset.sort_values('random')
                            del test_dataset['random']
                            input_size = train_dataset.shape[1]-1
                            
                            train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)
                            if n < 50000:
                                train_dataset = ut.down_sampling(train_dataset)
                                train_dataset = train_dataset.sample(n)
                            else: 
                                trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                n2 = n - trainig_dataset_a.shape[0]
                                trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])
                            
                            train_dataset = ut.sort_columns(train_dataset)
                            test_dataset = ut.sort_columns(test_dataset)
                            test_dataset_pred = test_dataset.copy()
                            train_dataset_pred = train_dataset.copy()

                            data_prior = ut.generate_samples_2D(samples, train_dataset, distribution='gaussian')
                            if train_dataset[train_dataset.label=='ClassB'].shape[0] >= samples:
                                samples_prior = samples 
                            else: 
                                samples_prior = train_dataset[train_dataset.label=='ClassB'].shape[0] 

                            data_prior = pd.concat([data_prior, train_dataset[train_dataset.label=='ClassB'].sample(samples_prior)])


                            train_dataset, test_dataset, data_prior = ut.normalize(train_dataset, test_dataset, data_prior)

                            train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2)

                            train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2)
                            _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                            _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                            _, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
                            _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
                            _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

                            net = Net(input_size, hidden_size, hidden_size, num_classes)
                            net.cuda()

                            hist_val, hist_train, _ = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                            EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated, model_number=t, size=n)

                            acc_train, recall_train, f1_train = nn.get_results(net, train_loader, input_size)
                            acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
                            roc_train = nn.get_roc_curve(net, train_loader, input_size)
                            roc_test = nn.get_roc_curve(net, test_loader, input_size)
                            results.append([acc_train, acc_test,recall_train, recall_test, f1_train, f1_test, roc_train, roc_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt])

                            pd.DataFrame(results, columns=['acc_train', 'acc_test','recall_train', 'recall_test','f1_train', 'f1_test', 
                                   'roc_train', 'roc_test', 'epsilon', 'batch_size', 'hidden_size',
                             'aux_loss_activated', 'EPS1', 'n', 'opt']).to_csv('RRc_17_01_2023_subclasses_gaussian2d.csv')

In [ ]:
results = []

num_classes = 2
learning_rate = 0.005
samples = 3000
epsilon=0
#for epsilon in [0.1, 0.05, 0.025, 0.15]:
for batch_size in [256]:
    for hidden_size in [100]:
        for aux_loss_activated in [True, False]:
            for EPS1 in [0.025]:
                for n in [50000]:
                    for opt in [1]:
                        for t in range(10):
                            train_dataset, test_dataset = ut.load_files(dataset=1)
                            test_dataseta  = RRd_test[train_dataset.columns]
                            test_dataset = pd.concat([test_dataseta[test_dataseta.label=='ClassB'], test_dataseta])
                            test_dataset['random'] = np.random.random(test_dataset.shape[0])
                            test_dataset = test_dataset.sort_values('random')
                            del test_dataset['random']
                            input_size = train_dataset.shape[1]-1
                            
                            
                            train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)
                            if n < 50000:
                                train_dataset = ut.down_sampling(train_dataset)
                                train_dataset = train_dataset.sample(n)
                            else: 
                                trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                n2 = n - trainig_dataset_a.shape[0]
                                trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])
                            
                            train_dataset = ut.sort_columns(train_dataset)
                            test_dataset = ut.sort_columns(test_dataset)
                            test_dataset_pred = test_dataset.copy()
                            train_dataset_pred = train_dataset.copy()

                            data_prior = ut.generate_samples_2D(samples, train_dataset, distribution='gaussian')
                            if train_dataset[train_dataset.label=='ClassB'].shape[0] >= samples:
                                samples_prior = samples 
                            else: 
                                samples_prior = train_dataset[train_dataset.label=='ClassB'].shape[0] 

                            data_prior = pd.concat([data_prior, train_dataset[train_dataset.label=='ClassB'].sample(samples_prior)])


                            train_dataset, test_dataset, data_prior = ut.normalize(train_dataset, test_dataset, data_prior)

                            train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2)

                            train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2)
                            _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                            _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                            _, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
                            _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
                            _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

                            net = Net(input_size, hidden_size, hidden_size, num_classes)
                            net.cuda()

                            hist_val, hist_train, _ = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                            EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated, model_number=t, size=n)

                            acc_train, recall_train, f1_train = nn.get_results(net, train_loader, input_size)
                            acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
                            roc_train = nn.get_roc_curve(net, train_loader, input_size)
                            roc_test = nn.get_roc_curve(net, test_loader, input_size)
                            results.append([acc_train, acc_test,recall_train, recall_test, f1_train, f1_test, roc_train, roc_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt])

                            pd.DataFrame(results, columns=['acc_train', 'acc_test','recall_train', 'recall_test','f1_train', 'f1_test', 
                                   'roc_train', 'roc_test', 'epsilon', 'batch_size', 'hidden_size',
                             'aux_loss_activated', 'EPS1', 'n', 'opt']).to_csv('RRd_17_01_2023_subclasses_gaussian2d.csv')

In [ ]:
results = []

num_classes = 2
learning_rate = 0.005
samples = 3000
epsilon=0
#for epsilon in [0.1, 0.05, 0.025, 0.15]:
for batch_size in [256]:
    for hidden_size in [100]:
        for aux_loss_activated in [True, False]:
            for EPS1 in [0.025]:
                for n in [50000]:
                    for opt in [1]:
                        for t in range(10):
                            
                            train_dataset, test_dataset = ut.load_files(dataset=1)
                            test_dataseta  = RRe_test[train_dataset.columns]
                            test_dataset = pd.concat([test_dataseta[test_dataseta.label=='ClassB'], test_dataseta])
                            test_dataset['random'] = np.random.random(test_dataset.shape[0])
                            test_dataset = test_dataset.sort_values('random')
                            del test_dataset['random']
                            input_size = train_dataset.shape[1]-1
                            
                            input_size = train_dataset.shape[1]-1
                            train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)
                            if n < 50000:
                                train_dataset = ut.down_sampling(train_dataset)
                                train_dataset = train_dataset.sample(n)
                            else: 
                                trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                n2 = n - trainig_dataset_a.shape[0]
                                trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])
                            
                            train_dataset = ut.sort_columns(train_dataset)
                            test_dataset = ut.sort_columns(test_dataset)
                            test_dataset_pred = test_dataset.copy()
                            train_dataset_pred = train_dataset.copy()

                            data_prior = ut.generate_samples_2D(samples, train_dataset, distribution='gaussian')
                            if train_dataset[train_dataset.label=='ClassB'].shape[0] >= samples:
                                samples_prior = samples 
                            else: 
                                samples_prior = train_dataset[train_dataset.label=='ClassB'].shape[0] 

                            data_prior = pd.concat([data_prior, train_dataset[train_dataset.label=='ClassB'].sample(samples_prior)])


                            train_dataset, test_dataset, data_prior = ut.normalize(train_dataset, test_dataset, data_prior)

                            train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2)

                            train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2)
                            _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                            _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                            _, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
                            _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                            _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
                            _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

                            net = Net(input_size, hidden_size, hidden_size, num_classes)
                            net.cuda()

                            hist_val, hist_train, _ = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                            EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated, model_number=t, size=n)

                            acc_train, recall_train, f1_train = nn.get_results(net, train_loader, input_size)
                            acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
                            roc_train = nn.get_roc_curve(net, train_loader, input_size)
                            roc_test = nn.get_roc_curve(net, test_loader, input_size)
                            results.append([acc_train, acc_test,recall_train, recall_test, f1_train, f1_test, roc_train, roc_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt])

                            pd.DataFrame(results, columns=['acc_train', 'acc_test','recall_train', 'recall_test','f1_train', 'f1_test', 
                                   'roc_train', 'roc_test', 'epsilon', 'batch_size', 'hidden_size',
                             'aux_loss_activated', 'EPS1', 'n', 'opt']).to_csv('RRe_17_01_2023_subclasses_gaussian2d.csv')

In [ ]:
n = 50000
samples = 3000
for i in range(0,30):

    net = Net(input_size, hidden_size, hidden_size, num_classes)
    net.load_state_dict(dict(torch.load('model_'+str(i)+'_False_50000.pt'))['model'+str(i)+'_False_50000'])
    net.eval()
    net.cuda()

    train_dataset , test_dataset = ut.load_files(dataset=1)
    input_size = test_dataset.shape[1]-1
    train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)
    
    trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
    n2 = n - trainig_dataset_a.shape[0]
    trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
    train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])
    test_dataset = ut.sort_columns(test_dataset)
    test_dataset_pred = test_dataset.copy()
    data_prior = ut.generate_samples_2D(samples, train_dataset, distribution='gaussian')
    if train_dataset[train_dataset.label=='ClassB'].shape[0] >= samples:
        samples_prior = samples 
    else: 
        samples_prior = train_dataset[train_dataset.label=='ClassB'].shape[0] 

    data_prior = pd.concat([data_prior, train_dataset[train_dataset.label=='ClassB'].sample(samples_prior)])
    train_dataset, test_dataset, data_prior = ut.normalize(train_dataset, test_dataset, data_prior)

    _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
    _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

    acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
    roc_test = nn.get_roc_curve(net, test_loader, input_size)

In [ ]:
RRab_test